In [ ]:
import geopandas as gpd
import sentinelhub
import datetime

In [2]:
import sys
sys.path.append('..')

In [3]:
import mysecrets
import config
import cdseutils.utils
import cdseutils.sentinel2

In [4]:
cdse_creds = mysecrets.PROFILES['nikhilsasirajan@gmail.com']

In [5]:
roi_filepath = '../../ethiopia/data/Ethiopia_Bounding_Boxs/Ethiopia_2021_East_Bounding_Box.shp'

In [6]:
shapes_gdf = gpd.read_file(roi_filepath).to_crs('epsg:4326')

In [7]:
startdate = datetime.datetime(2021, 3, 1)
enddate = datetime.datetime(2021, 5, 1)

In [ ]:
bboxes = cdseutils.utils.get_bboxes(shapes_gdf=shapes_gdf)

In [9]:
catalog_save_folderpath = '../data/fetch_catalog_cache'

In [15]:
# fetch_catalog is caching the results, cache_folderpath is a
# critical parameter to reduce api calls.
catalog_gdf, results = cdseutils.utils.fetch_catalog(
    bboxes = bboxes,
    sh_creds = cdse_creds.sh_creds,
    collection = sentinelhub.DataCollection.SENTINEL2_L2A,
    startdate = startdate,
    enddate = enddate,
    cache_folderpath = catalog_save_folderpath,
)
catalog_gdf['cloud_cover'] = [x['properties']['eo:cloud_cover'] for x in results]

In [ ]:
catalog_gdf

In [ ]:
catalog_gdf['s3url'][0]

In [ ]:
s3paths, download_filepaths = \
cdseutils.sentinel2.get_s3paths(
    s3urls = catalog_gdf['s3url'],
    s3_creds = cdse_creds.s3_creds,
    root_folderpath = config.FOLDERPATH_SATELLITE,
    bands = cdseutils.constants.Bands.S2L2A.ALL,
    satellite = cdseutils.constants.Bands.S2L2A.NAME,
)

In [ ]:
s3paths